# Wrangle and Analyze Data of a Twitter Account


## Table of Contents
- [1. Introduction](#intro)
- [2. Gather data](#gather)
- [3. Assess data](#assess)
- [4. Clean data](#clean)
- [5. Store](#store)


<a id='gather'></a>
## 1. Introduction

This project is an analysis of correlation between the Covid-19 cases and the political environment of different countries. Goal is to find answers or at least indicators to questions like: 
- Did the countries which had more success in containing the amount of Covid-19 cases something in common? 
- Is there a correlation in  Gross domestic product, Human Development Index or political ideology with the amount of Covid-19 cases of the country.

Main goal of this project is to generate a comprehensive exploratory and explanatory data analysis of the gathered data. The data analysis process is distributed over three ipynb-files: gather_clean_Covid19.ipynb, exploration_Covid19.ipynb and slide_deck_Covid19.ipynb.

Firstly, as part of gather_clean_Covid19.ipynb data is gathered from different sources: The Covid-19 data of this project is retrieved via programmatically downloaded csv-files from the GitHub repository [Covid-19](https://github.com/CSSEGISandData/COVID-19) and additional data about countries is retrieved via the wikipedia API. Secondly, the data from the different sources is visually and programmatically assessed to be cleaned.
The exploratory and explanatory data analysis of the gathered data is performed in exploration_Covid19.ipynb. Finally the findings are presented in slide_deck_Covid19.ipynb.

In [932]:
# Import necessary libraries
import numpy as np
import pandas as pd
from datetime import date
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import os # to work with local directory
import re
import wptools
import json # to create json file from python dictionary
import time # for timer 
sns.set()

<a id='intro'></a>
## 2. Gather data

####  Data is gathered from three different sources of data as described in steps below:

1. Fatality, confirmed cases, recovered cases and data by country is retrieved via programmatically downloaded csv-files from the GitHub repository [Covid-19](https://github.com/CSSEGISandData/COVID-19).
2. Additional data is retrieved via the wptools API from different wikipedia articles.

### a. Read data from programmatically download csv-file

In [933]:
# Gather data from John Hopkins GitHub 
df_JHU_Fatality = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
df_JHU_Confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df_JHU_Recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
df_JHU_Countries = pd.read_csv('https://raw.githubusercontent.com/RRighart/covid-19/master/countries.csv')

In [934]:
df_OWID_Covid = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')
df_OWID_Testing = pd.read_csv('https://covid.ourworldindata.org/data/testing/covid-testing-latest-data-source-details.csv')
df_OWID_Countries = pd.read_csv('https://covid.ourworldindata.org/data/ecdc/locations.csv')

### b. Read data from local datasets

Data downloaded manually from different databases, [European statistical database](https://ec.europa.eu/eurostat/data/database), [Wikipedia table on intensive care units](https://en.wikipedia.org/wiki/List_of_countries_by_hospital_beds) and [United Nations database](https://data.un.org):

In [935]:
df_ESTAT_census = pd.read_csv('inputData/Eurostat_census_2001.csv')
df_WIKI_ICU = pd.read_csv('inputData/Wikipedia_ICU.csv')
df_UN_births = pd.read_csv('inputData/UNdata_birthsByMonth.csv')
df_UN_deaths = pd.read_csv('inputData/UNdata_deathsByMonth.csv')

### c. Query additional information for countries via wikipedia API

Additional Information
- Leader Gender
- Ideology of Leading Party
- Amount of Intensive Care Beds
- Gross domestic product per capita
- Human Development Index

In [936]:
# Query for every tweet id in enhanced twitter archive and save tweet-information in json-format to 'tweet_json.txt'
'''             
country_jsons = {}
county_id_errors = []
start = time.time()
count = 0


with open('country_json.txt', 'w') as outfile:
    
    for country in df_JHU_Countries['Country/Region']:
        count +=1
        try:
            # Query API for data of wikipedia article
            article = wptools.page(country).get_parse()
            infobox = article.data['infobox']
            # Measure elapsed time
            mid_s = time.time()
            # Print id and time elapsed
            print(str(count) + str(mid_s - start) )
            # Write json of tweet to 'tweet_json.txt'
            json.dump(infobox, outfile)
            # New line
            outfile.write("\n")

        # Not best practice to catch all exceptions but fine for this short script
        except Exception as error:
            mid_f = time.time()
            print(str(count) + str(mid_f - start) + str(error))
            # Gather ids of id's without status
            tweet_id_errors.append([count, str(tweet_id)])
            
    end = time.time()
    print(end - start)
    
    '''

'             \ncountry_jsons = {}\ncounty_id_errors = []\nstart = time.time()\ncount = 0\n\n\nwith open(\'country_json.txt\', \'w\') as outfile:\n    \n    for country in df_JHU_Countries[\'Country/Region\']:\n        count +=1\n        try:\n            # Query API for data of wikipedia article\n            article = wptools.page(country).get_parse()\n            infobox = article.data[\'infobox\']\n            # Measure elapsed time\n            mid_s = time.time()\n            # Print id and time elapsed\n            print(str(count) + str(mid_s - start) )\n            # Write json of tweet to \'tweet_json.txt\'\n            json.dump(infobox, outfile)\n            # New line\n            outfile.write("\n")\n\n        # Not best practice to catch all exceptions but fine for this short script\n        except Exception as error:\n            mid_f = time.time()\n            print(str(count) + str(mid_f - start) + str(error))\n            # Gather ids of id\'s without status\n       

In [937]:
'''
so = wptools.page('Germany').get_parse()
infobox = so.data['infobox']
print(infobox)
'''

"\nso = wptools.page('Germany').get_parse()\ninfobox = so.data['infobox']\nprint(infobox)\n"

<a id='assess'></a>
## 3. Assess data

After gathering each of the above pieces of data, they are assessed visually and programmatically for quality and tidiness issues. Requirements to be met:

- Quality requirements:
    - Completeness: All necessary records in dataframes, no specific rows, columns or cells missing.
    - Validity: No records available, that do not conform schema.
    - Accuracy: No wrong data, that is valid.
    - Consistency: No data, that is valid and accurate, but referred to in multiple correct ways.
- Tidiniss requirements (as defined by Hadley Wickham):
    - each variable is a column
    - each observation is a row
    - each type of observational unit is a table.

### a. Visual assessment

In [938]:
# Check layout of df_JHU_Countries vsiually
df_JHU_Countries.sample(n=5)

,Unnamed: 0,Country/Region,inhabitants,area
16,16,Poland,37864109,312685
10,10,Austria,8999973,83858
15,15,Luxembourg,625978,2590
25,25,Taiwan,23816775,35410
3,3,Belgium,11579502,30510


In [939]:
# Check layout of df_JHU_Fatality vsiually
df_JHU_Fatality.sample(n=5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
37,Grand Princess,Canada,37.648900,-122.665500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,NaN,Dominican Republic,18.735700,-70.162700,0,0,0,0,0,0,...,373,380,385,388,393,402,409,422,424,428
91,NaN,Czechia,49.817500,15.473000,0,0,0,0,0,0,...,270,273,276,280,282,283,290,293,295,296
243,NaN,Saint Kitts and Nevis,17.357822,-62.782998,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44,Quebec,Canada,52.939900,-73.549100,0,0,0,0,0,0,...,2632,2726,2787,2929,3014,3132,3221,3352,3402,3484


In [940]:
# Check layout of df_JHU_Confirmed vsiually
df_JHU_Confirmed.sample(n=5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,36,43,43,45,45,45,45,48,48,48
87,NaN,Croatia,45.1000,15.2000,0,0,0,0,0,0,...,2125,2161,2176,2187,2196,2207,2213,2221,2222,2224
84,NaN,Congo (Kinshasa),-4.0383,21.7587,0,0,0,0,0,0,...,863,937,937,991,1024,1102,1169,1242,1298,1455
177,NaN,Pakistan,30.3753,69.3451,0,0,0,0,0,0,...,24644,26435,28736,30334,32081,34336,35298,35788,38799,38799
152,NaN,Madagascar,-18.7669,46.8691,0,0,0,0,0,0,...,193,193,193,193,186,186,212,230,238,283


In [941]:
# Check layout of df_JHU_Recovered vsiually
df_JHU_Recovered.sample(n=5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
181,NaN,Portugal,39.3999,-8.2245,0,0,0,0,0,0,...,2258,2422,2499,2549,2549,3013,3182,3198,3328,3822
246,NaN,South Sudan,6.8770,31.3070,0,0,0,0,0,0,...,0,2,2,2,2,2,2,3,4,4
203,NaN,Sweden,63.0000,16.0000,0,0,0,0,0,0,...,4971,4971,4971,4971,4971,4971,4971,4971,4971,4971
184,NaN,Russia,60.0000,90.0000,0,0,0,0,0,0,...,23803,26608,31916,34306,39801,43512,48003,53530,58226,63166
30,NaN,Brunei,4.5353,114.7277,0,0,0,0,0,0,...,131,132,132,134,134,134,134,134,135,136


In [942]:
# Check layout of df_OWID_Covid vsiually
df_OWID_Covid.sample(n=5)

# df_OWID_Covidchange 'location' to 'country'
# df_OWID_Covid create df_OWID_Countries with 'iso_code', 'location', 'population', 'population_density', 'median_age', 'aged_65_older', 'aged_70_older', 'gdp_per_capita', 'diabetes_prevalence', 'female_smokers', 'male_smokers', 'handwashing_facilities', 'hospital_beds_per_100k'
# df_OWID_Covid merge it to df_country

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
4097,CZE,Czech Republic,2020-05-05,7819,38,252,4,730.135,3.548,23.532,...,19.027,11.580,32605.906,NaN,227.485,6.82,30.5,38.3,NaN,6.63
10379,LUX,Luxembourg,2020-04-09,3034,64,46,2,4846.831,102.240,73.485,...,14.312,9.842,94277.965,0.2,128.275,4.42,20.9,26.0,NaN,4.51
13912,QAT,Qatar,2020-02-18,0,0,0,0,0.000,0.000,0.000,...,1.307,0.617,116935.600,NaN,176.690,16.52,0.8,26.9,NaN,1.20
3810,CUW,Curacao,2020-04-13,14,0,1,0,85.314,0.000,6.094,...,16.367,10.068,NaN,NaN,NaN,11.62,NaN,NaN,NaN,NaN
11417,MNG,Mongolia,2020-03-26,10,0,0,0,3.050,0.000,0.000,...,4.031,2.421,11840.846,0.5,460.043,4.82,5.5,46.5,71.18,7.00


In [943]:
# Check layout of df_OWID_Testing vsiually
df_OWID_Testing.sample(n=5)

# df_OWID_Testing drop columns 'source URL', 'Source label', 'Notes', 'Number of observations', 'Daily change in cumulative total', 'Daily change in cumulative total per thousand', '3-day rolling mean daily change', '3-day rolling mean daily change per thousand', '7-day rolling mean daily change', '7-day rolling mean daily change per thousand','General source label', 'General source URL', 'Short description', 'Detailed description'
# df_OWID_Testing Either cut per regex country name from 'Entity' and rename country or join country name from other df

,ISO code,Entity,Date,Source URL,Source label,Notes,Number of observations,Cumulative total,Cumulative total per thousand,Daily change in cumulative total,Daily change in cumulative total per thousand,3-day rolling mean daily change,3-day rolling mean daily change per thousand,7-day rolling mean daily change,7-day rolling mean daily change per thousand,General source label,General source URL,Short description,Detailed description
49,MMR,Myanmar - samples tested,2020-05-16,https://mohs.gov.mm/Main/content/publication/2...,Myanmar Ministry of Health and Sports,This figure is taken from the interactive dash...,39,13634,0.251,NaN,NaN,NaN,NaN,NaN,NaN,Myanmar Ministry of Health and Sports,https://mohs.gov.mm/Home,The number of samples tested.,The Myanmar Ministry of Health and Sports prov...
26,GHA,Ghana - samples tested,2020-05-14,https://www.ghanahealthservice.org/covid19/arc...,Outbreak Response Management Situation Update,number of tests,9,172623,5.555,3938.0,0.127,3479.667,0.112,NaN,NaN,Outbreak Response Management,https://www.ghanahealthservice.org/covid19/,The units are unclear. Some press releases men...,Outbreak Response Management provides [daily s...
34,IRN,Iran - tests performed,2020-05-16,http://web.archive.org/web/20200516191022/http...,Government of Iran,NaN,35,672679,8.009,14075.0,0.168,14381.667,0.172,NaN,NaN,Government of Iran,http://irangov.ir/,The number of tests performed.,The Government of Iran provides daily press re...
22,FIN,Finland - samples tested,2020-05-15,https://experience.arcgis.com/experience/d40b2...,Finnish Department of Health and Welfare,NaN,78,143805,25.954,NaN,0.123,NaN,0.472,NaN,0.472,Finnish Department of Health and Welfare COVID...,https://experience.arcgis.com/experience/d40b2...,The number of samples tested.,The Finnish Department of Health and Welfare p...
63,ROU,Romania - tests performed,2020-05-14,https://github.com/adrianp/covid19romania,Ministry of Internal Affairs,Made available by Adrian-Tudor Panescu on Github,64,286217,14.878,8413.0,0.437,7999.333,0.416,8514.857,0.442,Ministry of Internal Affairs,https://github.com/adrianp/covid19romania,The number of tests performed.,Data is collected and made available [on GitHu...


In [944]:
# Check layout of df_OWID_Countries vsiually
df_OWID_Countries.sample(n=5)

# df_OWID_Countries convert datatype population to integer
# df_OWID_Countries drop 'countriesAndTerritories', 'population_year'

,countriesAndTerritories,location,continent,population_year,population
69,Finland,Finland,Europe,2020.0,5540718.0
52,Czechia,Czech Republic,Europe,2020.0,10708982.0
15,Bangladesh,Bangladesh,Asia,2020.0,164689383.0
118,Madagascar,Madagascar,Africa,2020.0,27691019.0
83,Guernsey,Guernsey,Europe,NaN,NaN


In [945]:
# Check layout of df_ESTAT_census vsiually
df_ESTAT_census.sample(n=5)

# df_ESTAT_census make columns from values in n_person
# df_ESTAT_census replace 'Germany (until 1990 former territory of the FRG)' with 'Germany'
# df_ESTAT_census drop country 'Bulgaria'
# df_ESTAT_census drop '4 persons', '5 persons', '6 persons or more'

,N_PERSON,GEO,TIME,AGE,HHCOMP,UNIT,Value,Flag and Footnotes
231,6 persons or more,Netherlands,2001,65 years or over,Total,Number,:,NaN
38,No persons,Latvia,2001,65 years or over,Total,Number,"593,765",NaN
236,6 persons or more,Slovenia,2001,65 years or over,Total,Number,:,NaN
142,4 persons,Spain,2001,65 years or over,Total,Number,:,NaN
212,5 persons,United Kingdom,2001,65 years or over,Total,Number,:,NaN


In [946]:
df_ESTAT_census.AGE.value_counts()

65 years or over    243
Name: AGE, dtype: int64

In [947]:
# Check layout of df_WIKI_ICU vsiually
df_WIKI_ICU.sample(n=5)

,country,continent,hospital_beds_per_1000_people,occupancy,ICU-CCB_beds_per_1000_people,ventilators
37,Chile,South America,2.11,79.1,NaN,NaN
5,Hungary,Europe,7.02,65.5,13.8,"2,560"
17,Slovenia,Europe,4.50,69.5,6.4,NaN
36,Sweden,Europe,2.22,NaN,5.8,570
40,Mexico,North America,1.38,74.0,1.2,"2,050"


In [948]:
df_UN_births.sample(n=5)
# Drop columns 'Area', 'Record Type', 'Reliability', 'Value Footnotes', 'Source Year'
# change datatype of columns  'Value' to integer
# Merge df_UN_births and df_UN_deaths on Year


,Country or Area,Year,Area,Month,Record Type,Reliability,Source Year,Value,Value Footnotes
2790,Czechia,2011,Total,February,Data tabulated by year of occurrence,"Final figure, complete",2012.0,8525.0,NaN
7839,Qatar,2016,Total,January,Data tabulated by year of occurrence,"Final figure, complete",2018.0,2265.0,NaN
6637,Netherlands,2017,Total,May,Data tabulated by year of occurrence,"Final figure, complete",2019.0,14367.0,30
6472,Montenegro,2013,Total,August,Data tabulated by year of occurrence,"Final figure, complete",2015.0,701.0,NaN
7972,Republic of Korea,2013,Total,Total,Data tabulated by year of occurrence,"Final figure, complete",2019.0,436455.0,33


In [949]:
df_UN_births.Area.value_counts()

Total    10373
Name: Area, dtype: int64

In [950]:
df_UN_deaths.sample(n=5)

,Country or Area,Year,Area,Month,Record Type,Reliability,Source Year,Value,Value Footnotes
2823,Egypt,2011,Total,September,Data tabulated by year of occurrence,"Final figure, complete",2013.0,35308.0,NaN
2745,Egypt,2017,Total,September,Data tabulated by year of occurrence,"Final figure, complete",2018.0,40863.0,NaN
5702,Mauritius,2016,Total,February,Data tabulated by year of registration,"Final figure, complete",2017.0,825.0,24
6287,Netherlands,2014,Total,April,Data tabulated by year of occurrence,"Final figure, complete",2016.0,11258.0,27
2961,Estonia,2014,Total,April,Data tabulated by year of occurrence,"Final figure, complete",2016.0,1302.0,NaN


### b. Programmatic assessment

In [951]:
# List of countries that are avaoilable in John Hopkins Dataset
df_JHU_Recovered['Country/Region'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Brazil', 'Brunei',
       'Bulgaria', 'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon',
       'Canada', 'Central African Republic', 'Chad', 'Chile', 'China',
       'Colombia', 'Congo (Brazzaville)', 'Congo (Kinshasa)',
       'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Diamond Princess',
       'Cuba', 'Cyprus', 'Czechia', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia',
       'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia',
       'Germany', 'Ghana', 'Grenada', 'Greece', 'Guatemala', 'Guinea',
       'Guinea-Bissau', 'Guyana', 'Haiti', 'H

In [952]:
# List of countries that are avaoilable in John Hopkins Dataset
df_OWID_Covid['location'].unique()

array(['Aruba', 'Afghanistan', 'Angola', 'Anguilla', 'Albania', 'Andorra',
       'United Arab Emirates', 'Argentina', 'Armenia',
       'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan',
       'Burundi', 'Belgium', 'Benin', 'Bonaire Sint Eustatius and Saba',
       'Burkina Faso', 'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas',
       'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda',
       'Bolivia', 'Brazil', 'Barbados', 'Brunei', 'Bhutan', 'Botswana',
       'Central African Republic', 'Canada', 'Switzerland', 'Chile',
       'China', "Cote d'Ivoire", 'Cameroon',
       'Democratic Republic of Congo', 'Congo', 'Colombia', 'Comoros',
       'Cape Verde', 'Costa Rica', 'Cuba', 'Curacao', 'Cayman Islands',
       'Cyprus', 'Czech Republic', 'Germany', 'Djibouti', 'Dominica',
       'Denmark', 'Dominican Republic', 'Algeria', 'Ecuador', 'Egypt',
       'Eritrea', 'Western Sahara', 'Spain', 'Estonia', 'Ethiopia',
       'Finland', 'Fiji', 'Falkland Islands', 'France',

In [953]:
# Available variables in dataset
list(df_OWID_Covid)

['iso_code',
 'location',
 'date',
 'total_cases',
 'new_cases',
 'total_deaths',
 'new_deaths',
 'total_cases_per_million',
 'new_cases_per_million',
 'total_deaths_per_million',
 'new_deaths_per_million',
 'total_tests',
 'new_tests',
 'total_tests_per_thousand',
 'new_tests_per_thousand',
 'tests_units',
 'population',
 'population_density',
 'median_age',
 'aged_65_older',
 'aged_70_older',
 'gdp_per_capita',
 'extreme_poverty',
 'cvd_death_rate',
 'diabetes_prevalence',
 'female_smokers',
 'male_smokers',
 'handwashing_facilities',
 'hospital_beds_per_100k']

In [954]:
df_OWID_Covid.query('location == "Germany" and date == "2020-05-13"')

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
4244,DEU,Germany,2020-05-13,171306,798,7634,101,2044.616,9.524,91.115,...,21.453,15.957,45229.245,NaN,156.139,8.31,28.2,33.1,NaN,8.0


In [955]:
df_Check = df_JHU_Confirmed.copy()
df_Check.rename(columns={'Country/Region': 'country'}, inplace=True)
df_Check.query('country == "Germany"')

,Province/State,country,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
120,NaN,Germany,51.0,9.0,0,0,0,0,0,1,...,169430,170588,171324,171879,172576,173171,174098,174478,175233,175752


In [956]:
# Check for countries which are referred to by different names in different dataframes
c_df_JHU_Fatality =  df_JHU_Fatality['Country/Region'].unique()
c_df_JHU_Confirmed = df_JHU_Confirmed['Country/Region'].unique()
c_df_JHU_Recovered = df_JHU_Recovered['Country/Region'].unique()
c_df_JHU_Countries = df_JHU_Countries['Country/Region'].unique()
c_df_OWID_Covid = df_OWID_Covid['location'].unique()
c_df_OWID_Testing = df_OWID_Testing['Entity'].unique()
c_df_OWID_Countries = df_OWID_Countries['location'].unique()
c_df_WIKI_ICU = df_WIKI_ICU['country'].unique()
c_df_UN_births = df_UN_births['Country or Area'].unique()
c_df_UN_deaths = df_UN_deaths['Country or Area'].unique()

all_country_names = list(c_df_JHU_Fatality) + list(c_df_JHU_Confirmed) + list(c_df_JHU_Recovered) + list(c_df_JHU_Countries) + list(c_df_OWID_Covid) + list(c_df_OWID_Testing) + list(c_df_OWID_Countries) + list(c_df_WIKI_ICU) + list(c_df_UN_births) + list(c_df_UN_deaths)

all_country_names = pd.Series(all_country_names).unique()

In [957]:
all_country_names

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Diamond Princess', 'Cuba', 'Cyprus',
       'Czechia', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Guinea', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary',
       'Iceland', 'India

### Findings, which contradict requirements:

#### Quality Observations:
- Validity: Some observations/rows in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality' contain the values for a region, for example Australia appears multiple times in column country as the observations are per region.
- Consistency: Data about Covid-19 cases differs slightly between John Hopkins and OWID, data which is available in both datasets will be kept only from John Hopkins

#### Tidiness Observations:
- The data of 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality' should be one observational unit 'df_covid' with columns 'country', 'date', 'recovered', 'confirmed', 'fatal' and 'date' beeing of type datetime.
- Column 'Country/Region' should only contain countries, therefore column name should by 'country', same for OWID data.
- Columns 'Province/State', 'Lat' and 'Long' are not necessary in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality'
- Data for countries, which are not of interested is not needed in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality', 'df_JHU_Countries'
- In the df_OWID_Covid_clean dataframes there is covid-related data where the variation frequency is daily and there is data not directly covid-related where data variation frequency is monthly or even constant for . Thus, there should be three observational units, df_covid for covid-related data and daily observations, df_OWID_country.




- df_OWID_Countries convert datatype population to integer
- df_OWID_Countries drop 'countriesAndTerritories', 'population_year'
- df_ESTAT_census make columns from values in n_person
- df_ESTAT_census replace 'Germany (until 1990 former territory of the FRG)' with 'Germany'
- df_ESTAT_census drop country 'Bulgaria'
- df_ESTAT_census drop '4 persons', '5 persons', '6 persons or more'

-  merge df_OWID_Countries with df_country

<a id='clean'></a>
## 4. Clean data

In [958]:
# Create copies for cleaning process to preserve original dataframes
df_JHU_Fatality_clean = df_JHU_Fatality.copy()
df_JHU_Confirmed_clean = df_JHU_Confirmed.copy()
df_JHU_Recovered_clean = df_JHU_Recovered.copy()
df_JHU_Countries_clean = df_JHU_Countries.copy()
df_OWID_Covid_clean = df_OWID_Covid.copy()
df_OWID_Testing_clean = df_OWID_Testing.copy()
df_OWID_Countries_clean = df_OWID_Countries.copy()
df_ESTAT_census_clean = df_ESTAT_census.copy()
df_WIKI_ICU_clean = df_WIKI_ICU.copy()
df_UN_births_clean = df_UN_births.copy()
df_UN_deaths_clean = df_UN_deaths.copy()

### Issue 1:
#### Observe:
-  Tidiness: Columns 'Province/State', 'Lat' and 'Long' are not necessary in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality'

#### Define:
- Drop columns 'Province/State', 'Lat' and 'Long'

#### Code:

In [959]:
# Drop variables which are only necessary for retweets
df_JHU_Fatality_clean.drop(['Province/State', 'Lat', 'Long'], axis=1, inplace=True)
df_JHU_Confirmed_clean.drop(['Province/State', 'Lat', 'Long'], axis=1, inplace=True)
df_JHU_Recovered_clean.drop(['Province/State', 'Lat', 'Long'], axis=1, inplace=True)

#### Test:

In [960]:
# Check if columnns 'Province/State', 'Lat' and 'Long' dropped
df_JHU_Fatality_clean.head(1)

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,106,109,115,120,122,127,132,136,153,168


In [961]:
# Check if columnns 'Province/State', 'Lat' and 'Long' dropped
df_JHU_Confirmed_clean.head(1)

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,3563,3778,4033,4402,4687,4963,5226,5639,6053,6402


In [962]:
# Check if columnns 'Province/State', 'Lat' and 'Long' dropped
df_JHU_Recovered_clean.head(1)

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,468,472,502,558,558,610,648,691,745,745


### Issue 2:
#### Observe:
- Tidiness: Column 'Country/Region' should only contain countries, therefore column name should by 'Country'.

#### Define:
- Rename column 'Country/Region' to 'country'

#### Code:

In [963]:
# Rename coloumn inplace to identic primary key
df_JHU_Fatality_clean.rename(columns={'Country/Region': 'country'}, inplace=True)
df_JHU_Confirmed_clean.rename(columns={'Country/Region': 'country'}, inplace=True)
df_JHU_Recovered_clean.rename(columns={'Country/Region': 'country'}, inplace=True)
df_JHU_Countries_clean.rename(columns={'Country/Region': 'country'}, inplace=True)
df_OWID_Covid_clean.rename(columns={'location': 'country'}, inplace=True)
df_OWID_Countries_clean.rename(columns={'location': 'country'}, inplace=True)

#### Test:

In [964]:
assert df_JHU_Fatality_clean.country.any()

In [965]:
assert df_JHU_Confirmed_clean.country.any()

In [966]:
assert df_JHU_Recovered_clean.country.any()

In [967]:
assert df_JHU_Countries_clean.country.any()

In [968]:
assert df_OWID_Covid_clean.country.any()

In [969]:
assert df_OWID_Countries_clean.country.any()

### Issue 3:
#### Observe:
- Tidiness: In the df_OWID_Covid_clean dataframes there is covid-related data where the variation frequency is daily and there is data not directly covid-related where data variation frequency is monthly or even constant for . Thus, there should be three observational units, df_covid for covid-related data and daily observations, df_OWID_country.

#### Define
- Create new dataframe df_OWID_Countries with columns 'iso_code', 'location', 'population', 'population_density', 'median_age', 'aged_65_older', 'aged_70_older', 'gdp_per_capita', 'diabetes_prevalence', 'female_smokers', 'male_smokers', 'handwashing_facilities', 'hospital_beds_per_100k'.

#### Code:

In [970]:
df_OWID_Countries = df_OWID_Covid_clean.copy()
df_OWID_Countries.drop([ 'date',
                         'total_cases',
                         'new_cases',
                         'total_deaths',
                         'new_deaths',
                         'total_cases_per_million',
                         'new_cases_per_million',
                         'total_deaths_per_million',
                         'new_deaths_per_million',
                         'total_tests',
                         'new_tests',
                         'total_tests_per_thousand',
                         'new_tests_per_thousand',
                         'tests_units',
                         'cvd_death_rate',
                         'handwashing_facilities',
                         'extreme_poverty'], axis=1, inplace=True)
df_OWID_Countries = df_OWID_Countries.drop_duplicates()

#subset=['A', 'C'], 

#### Test:

In [971]:
df_OWID_Countries.country.value_counts().head(3)

Mexico    1
Iran      1
Guinea    1
Name: country, dtype: int64

In [972]:
list(df_OWID_Countries)

['iso_code',
 'country',
 'population',
 'population_density',
 'median_age',
 'aged_65_older',
 'aged_70_older',
 'gdp_per_capita',
 'diabetes_prevalence',
 'female_smokers',
 'male_smokers',
 'hospital_beds_per_100k']

### Issue 4:
#### Observe:
- Validity: Some observations/rows in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality' contain the values for a region, for example Australia appears multiple times in column country as the observations are per region.

#### Define: 
- Sum values of rows with same entry in column country by using groupby

#### Code:

In [973]:
# Groupby and sum
df_JHU_Fatality_clean = df_JHU_Fatality_clean.groupby(['country'], as_index=False).sum()
df_JHU_Confirmed_clean = df_JHU_Confirmed_clean.groupby(['country'], as_index=False).sum()
df_JHU_Recovered_clean = df_JHU_Recovered_clean.groupby(['country'], as_index=False).sum()

#### Test:

In [974]:
df_JHU_Fatality_clean.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
183    False
184    False
185    False
186    False
187    False
Length: 188, dtype: bool

### Issue 5:
#### Observe:
- Tidiness: The data of 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality' should be one observational unit 'df_covid' with columns 'country', 'date', 'recovered', 'confirmed', 'fatal' and 'date' beeing of type datetime.

#### Define:
- Melt date columns to one column 'date', transform date to type datetime and merge the three dataframes to ones dataframe 'df_covid' with sorted date values.

#### Code:

In [975]:
# Melt each dataframe so that results in columns: country,
df_JHU_Fatality_clean = pd.melt(df_JHU_Fatality_clean, id_vars = ['country'], var_name='date', value_name='fatal')
df_JHU_Confirmed_clean = pd.melt(df_JHU_Confirmed_clean, id_vars = ['country'], var_name='date', value_name='confirmed')
df_JHU_Recovered_clean = pd.melt(df_JHU_Recovered_clean, id_vars = ['country'], var_name='date', value_name='recovered')

In [976]:
# Convert new columns date to datetime
df_JHU_Fatality_clean.date=pd.to_datetime(df_JHU_Fatality_clean.date)
df_JHU_Confirmed_clean.date=pd.to_datetime(df_JHU_Confirmed_clean.date)
df_JHU_Recovered_clean.date=pd.to_datetime(df_JHU_Recovered_clean.date)

In [977]:
# Merge three covid dataframes to one
df_covid = pd.merge(df_JHU_Fatality_clean, df_JHU_Confirmed_clean, on=['country','date'])
df_covid = pd.merge(df_covid, df_JHU_Recovered_clean, on=['country','date'])

In [978]:
# Sort date values by date
df_covid = df_covid.sort_values(by='date', ascending=True)

#### Test:

In [979]:
list(df_covid)

['country', 'date', 'fatal', 'confirmed', 'recovered']

In [980]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21808 entries, 0 to 21807
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   country    21808 non-null  object        
 1   date       21808 non-null  datetime64[ns]
 2   fatal      21808 non-null  int64         
 3   confirmed  21808 non-null  int64         
 4   recovered  21808 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 1022.2+ KB


In [981]:
df_covid

,country,date,fatal,confirmed,recovered
0,Afghanistan,2020-01-22,0,0,0
120,Namibia,2020-01-22,0,0,0
121,Nepal,2020-01-22,0,0,0
122,Netherlands,2020-01-22,0,0,0
123,New Zealand,2020-01-22,0,0,0
...,...,...,...,...,...
21684,Gambia,2020-05-16,1,23,12
21685,Georgia,2020-05-16,12,683,419
21686,Germany,2020-05-16,7938,175752,152600
21677,Estonia,2020-05-16,63,1770,934


### Issue 6:
#### Observe:
- Consistency: Some country names differ from one dataframe to another 

#### Define:
- ...

#### Code

In [982]:
# Rename coloumn inplace
df_covid['country'].replace({'US': 'United States', 'Taiwan*': 'Taiwan'}, inplace=True)
df_JHU_Countries_clean['country'].replace({'US': 'United States', 'Taiwan*': 'Taiwan'}, inplace=True)



In [983]:
list(df_OWID_Testing_clean)

['ISO code',
 'Entity',
 'Date',
 'Source URL',
 'Source label',
 'Notes',
 'Number of observations',
 'Cumulative total',
 'Cumulative total per thousand',
 'Daily change in cumulative total',
 'Daily change in cumulative total per thousand',
 '3-day rolling mean daily change',
 '3-day rolling mean daily change per thousand',
 '7-day rolling mean daily change',
 '7-day rolling mean daily change per thousand',
 'General source label',
 'General source URL',
 'Short description',
 'Detailed description']

In [984]:
# Extract country name from column 'Entity' via regex
df_OWID_Testing_clean['country'] = df_OWID_Testing_clean.Entity.str.extract(
    '([A-Z][a-z]{0,10}( [A-Z][a-z]{0,10})?)', expand=True)[0]

In [985]:
df_UN_births_clean['country'] = df_UN_births_clean['Country or Area'].str.extract(
    '([A-Z][a-z]{0,10}( [A-Z][a-z]{0,10})?)', expand=True)[0]

df_UN_deaths_clean['country'] = df_UN_deaths_clean['Country or Area'].str.extract(
    '([A-Z][a-z]{0,10}( [A-Z][a-z]{0,10})?)', expand=True)[0]


In [ ]:
# Extract country name from column 'Entity' via regex
df_OWID_Testing_clean['country'] = df_OWID_Testing_clean.Entity.str.extract(
    '([A-Z][a-z]{0,10}( [A-Z][a-z]{0,10})?)', expand=True)[0]

In [995]:
df_WIKI_ICU_clean['country'].unique()

array(['\xa0Japan', '\xa0South Korea', '\xa0Russia', '\xa0Germany',
       '\xa0Austria', '\xa0Hungary', '\xa0Czech Republic', '\xa0Poland',
       '\xa0Lithuania', '\xa0France', '\xa0Slovakia', '\xa0Belgium',
       '\xa0Latvia', '\xa0Hong Kong', '\xa0Estonia', '\xa0Luxembourg',
       '\xa0\xa0Switzerland', '\xa0Slovenia', '\xa0China', '\xa0Greece',
       '\xa0Australia', '\xa0Norway', '\xa0Portugal', '\xa0Netherlands',
       '\xa0Finland', '\xa0Italy', '\xa0Iceland', '\xa0Israel',
       '\xa0Spain', '\xa0Ireland', '\xa0Turkey', '\xa0United States',
       '\xa0New Zealand', '\xa0Denmark', '\xa0United Kingdom(more)',
       '\xa0Canada', '\xa0Sweden', '\xa0Chile', '\xa0Colombia',
       '\xa0India', '\xa0Mexico', '\xa0Ukraine', '\xa0Bangladesh'],
      dtype=object)

In [986]:
list(df_UN_births_clean.country.unique())

['Islands',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Anguilla',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Belarus',
 'Belgium',
 'Bermuda',
 'Bosnia',
 'Botswana',
 'Brazil',
 'British Virgin',
 'Brunei Darussalam',
 'Bulgaria',
 'Canada',
 'Cayman Islands',
 'Chile',
 'China',
 'Cook Islands',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cura',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Egypt',
 'El Salvador',
 'Estonia',
 'Faroe Islands',
 'Finland',
 'France',
 'Georgia',
 'Germany',
 'Greece',
 'Greenland',
 'Guam',
 'Guatemala',
 'Guernsey',
 'Hungary',
 'Iceland',
 'Iran',
 'Ireland',
 'Isle',
 'Israel',
 'Italy',
 'Japan',
 'Kazakhstan',
 'Kuwait',
 'Kyrgyzstan',
 'Latvia',
 'Lebanon',
 'Liechtenste',
 'Lithuania',
 'Luxembourg',
 'Malaysia',
 'Maldives',
 'Malta',
 'Mauritius',
 'Mexico',
 'Mongolia',
 'Montenegro',
 'Montserrat',
 'Netherlands',
 'New Caledonia',
 'New Zealand',
 'North Macedonia',
 'Norway',
 'Oman',
 'Palau',
 'Panama',


#### Test:

In [987]:
# Check for countries which are referred to by different names in different dataframes
c_df_covid = df_covid['country'].unique()
c_df_JHU_Countries_clean = df_JHU_Countries_clean['country'].unique()
df_OWID_Countries = df_OWID_Countries['country'].unique()
c_df_OWID_Testing_clean = df_OWID_Testing_clean['country'].unique()
c_df_OWID_Countries_clean = df_OWID_Countries_clean['country'].unique()
c_df_WIKI_ICU_clean = df_WIKI_ICU_clean['country'].unique()
c_df_UN_births_clean = df_UN_births_clean['country'].unique()
c_df_UN_deaths_clean = df_UN_deaths_clean['country'].unique()

all_country_names = list(c_df_covid) + list(c_df_JHU_Countries_clean) + list(c_df_OWID_Covid_clean) + list(c_df_OWID_Testing_clean) + list(c_df_OWID_Countries_clean) + list(c_df_WIKI_ICU_clean) + list(c_df_UN_births_clean) + list(c_df_UN_deaths_clean)

all_country_names = pd.Series(all_country_names).unique()

In [988]:
all_country_names

array(['Afghanistan', 'Namibia', 'Nepal', 'Netherlands', 'New Zealand',
       'Nicaragua', 'Niger', 'Nigeria', 'North Macedonia', 'Norway',
       'Oman', 'Pakistan', 'Panama', 'Papua New Guinea', 'Paraguay',
       'Peru', 'Philippines', 'Poland', 'Portugal', 'Qatar', 'Mozambique',
       'Morocco', 'Montenegro', 'Mongolia', 'Latvia', 'Lebanon',
       'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania',
       'Luxembourg', 'MS Zaandam', 'Romania', 'Madagascar', 'Malaysia',
       'Maldives', 'Mali', 'Malta', 'Mauritania', 'Mauritius', 'Mexico',
       'Moldova', 'Monaco', 'Malawi', 'Rwanda', 'Saint Kitts and Nevis',
       'Saint Lucia', 'Thailand', 'Timor-Leste', 'Togo',
       'Trinidad and Tobago', 'Tunisia', 'Turkey', 'United States',
       'Uganda', 'Ukraine', 'Tanzania', 'United Arab Emirates', 'Uruguay',
       'Uzbekistan', 'Venezuela', 'Vietnam', 'West Bank and Gaza',
       'Western Sahara', 'Yemen', 'Zambia', 'Zimbabwe', 'United Kingdom',
       'Laos', 'Tajikis

In [989]:
len(countrie_in_both_df)

400

In [990]:
len(df_covid.country.unique())

188

In [991]:
len(df_OWID_Covid_clean.country.unique())

212

In [992]:
list(df_OWID_Covid_clean)

['iso_code',
 'country',
 'date',
 'total_cases',
 'new_cases',
 'total_deaths',
 'new_deaths',
 'total_cases_per_million',
 'new_cases_per_million',
 'total_deaths_per_million',
 'new_deaths_per_million',
 'total_tests',
 'new_tests',
 'total_tests_per_thousand',
 'new_tests_per_thousand',
 'tests_units',
 'population',
 'population_density',
 'median_age',
 'aged_65_older',
 'aged_70_older',
 'gdp_per_capita',
 'extreme_poverty',
 'cvd_death_rate',
 'diabetes_prevalence',
 'female_smokers',
 'male_smokers',
 'handwashing_facilities',
 'hospital_beds_per_100k']

In [993]:
df_OWID_Covid_clean.new_tests.value_counts()

1.0       14
2.0       13
8.0       13
16.0      12
10.0      11
          ..
1360.0     1
950.0      1
3680.0     1
2190.0     1
1856.0     1
Name: new_tests, Length: 2983, dtype: int64


- df_OWID_Testing drop columns 'source URL', 'Source label', 'Notes', 'Number of observations', 'Daily change in cumulative total', 'Daily change in cumulative total per thousand', '3-day rolling mean daily change', '3-day rolling mean daily change per thousand', '7-day rolling mean daily change', '7-day rolling mean daily change per thousand','General source label', 'General source URL', 'Short description', 'Detailed description'
- df_OWID_Testing Either cut per regex country name from 'Entity' and rename country or join country name from other df

<a id='store'></a>
## 5. Store clean data

In [994]:
# Store cleaned dataset to csv
df_covid.to_csv('covid_master.csv', encoding='utf-8')